In [1]:
import pandas as pd
import numpy as np
import requests # request to get the pdf
import pymupdf # a Python library that supports file formats like PDF, among others
import fitz

from PyPDF2 import PdfFileReader # Read and parse your content pdf
from bs4 import BeautifulSoup # for parse the html and find all url hrf with ".pdf" final

from langdetect import detect, detect_langs, lang_detect_exception #to recognize the language of the text
from collections import Counter

import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
def get_first_important_year(types, dates):
    try:
        types = types.split(';')
        dates = dates.split(';')
        # Create a list of (type, date) pairs
        type_date_pairs = list(zip(types, dates))

        # First look for 'Entered Into Force'
        for t, d in type_date_pairs:
            if t.strip() == 'Entered Into Force':
                dt = pd.to_datetime(d.strip(), errors='coerce')
                return int(dt.year) if pd.notnull(dt) else pd.NA

        # If not found, look for 'Passed/Approved'
        for t, d in type_date_pairs:
            if t.strip() == 'Passed/Approved':
                dt = pd.to_datetime(d.strip(), errors='coerce')
                return int(dt.year) if pd.notnull(dt) else pd.NA

        return pd.NA
    except:
        return pd.NA


In [3]:
cclw_data = pd.read_csv('../data/CCLW.csv')
cclw_data

,Document ID,Document Title,Family ID,Family Title,Family Summary,Collection Title(s),Collection Description(s),Document Variant,Document Content URL,Language,...,Document Type,Category,Framework,Topic/Response,Hazard,Sector,Keyword,Instrument,Author,Author Type
0,ordinance-730-03-on-the-encouragement-of-the-p...,Ordinance 730.03 on the encouragement of the p...,ordinance-730-03-on-the-encouragement-of-the-p...,Ordinance 730.03 on the encouragement of the p...,<p>This regulation defines basic determination...,NaN,NaN,Original Language,https://www.fedlex.admin.ch/filestore/fedlex.d...,French;German,...,Regulation,Executive,NaN,Mitigation,NaN,Energy,Renewables;Electricity;Heat,"Processes, plans and strategies|Governance",NaN,NaN
1,green-construction-through-wood-program-applic...,Green Construction through Wood Program - APPL...,green-construction-through-wood-program_d7f2,Green Construction through Wood Program,<p>The Green Construction through Wood (GCWood...,NaN,NaN,Original Language,https://ostr-backend-prod.azurewebsites.net/se...,English,...,Programme,Executive,NaN,Mitigation,NaN,Buildings,Energy Demand;Energy Efficiency;Mitigation,Provision of climate funds|Direct Investment,NaN,NaN
2,product-emissions-standards-act_2677,Product Emissions Standards Act,product-emissions-standards-act-2017_7c97,Product Emissions Standards Act (2017),<p>This act establishes a national framework t...,NaN,NaN,Original Language,https://www.legislation.gov.au/C2017A00104/lat...,English,...,Act,Legislative,NaN,Mitigation,NaN,Industry;Energy;Agriculture;LULUCF,Energy Efficiency;Energy Demand,"Standards, obligations and norms|Regulation",NaN,NaN
3,national-steel-policy_5eeb,National steel policy,national-steel-policy_42a5,National steel policy,<p>the Government has come out with National S...,NaN,NaN,Original Language,https://steel.gov.in/national-steel-policy-nsp...,English,...,Policy,Executive,NaN,Mitigation,NaN,Industry,Mitigation,"Standards, obligations and norms|Regulation;Pr...",NaN,NaN
4,notification-s-o-2670-e-fuel-consumption-stand...,Notification S.O. 2670(E) - Fuel-Consumption S...,fuel-consumption-standards-for-heavy-duty-vehi...,Fuel-Consumption Standards for Heavy-Duty Vehi...,<p>India initially introduced fuel consumption...,NaN,NaN,Original Language,https://www.transportpolicy.net/wp-content/upl...,English;Hindi,...,Rules,Executive,NaN,Mitigation,NaN,Transport,Energy Efficiency,"Standards, obligations and norms|Regulation",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8969,national-decarbonization-plan-government-of-co...,National Decarbonization Plan - Government of ...,national-decarbonization-plan-government-of-co...,National Decarbonization Plan - Government of ...,National Decarbonization Plan - Government of ...,NaN,NaN,Translation,https://unfccc.int/sites/default/files/resourc...,English,...,Long-Term Low-Emission Development Strategy,UNFCCC,NaN,NaN,NaN,NaN,NaN,NaN,Costa Rica,Party
8970,national-clean-development-strategy-2020-2050_...,National Clean Development Strategy 2020-2050,national-clean-development-strategy-2020-2050_...,National Clean Development Strategy 2020-2050,"National Clean Development Strategy 2020-2050,...",NaN,NaN,NaN,https://unfccc.int/sites/default/files/resourc...,English,...,Long-Term Low-Emission Development Strategy,UNFCCC,NaN,NaN,NaN,NaN,NaN,NaN,Hungary,Party
8971,namibia-first-ndc-updated-submission_3c8d,Namibia First NDC (Updated submission),namibia-nationally-determined-contribution-ndc...,Namibia Nationally Determined Contribution. N...,"<p>Namibia First NDC (Updated submission), Nat...",Namibia's Nationally Determined Contributions,NaN,NaN,https://unfccc.int/sites/default/files/NDC/202...,English,...,Nationally Determined Contribution,UNFCCC,NaN,NaN,NaN,NaN,NaN,NaN,Namibia,Party
8972,myanmar-first-ndc-archived_8559,Myanmar First NDC (Archived),myanmar-nationally-determined-contribution-ndc...,Myanmar Nationally Determined Contribution. ND...,"<

In [4]:
#cclw_data['Topic/Response'] = cclw_data['Topic/Response'].fillna('Unknown')
## Since we are only looking into the indicator variable, we are exploding the dataframe and do not focus on repeated values in other columns which come with explosion.
cclw_data['Topic/Response'] = cclw_data['Topic/Response'].str.split(';')
cclw_data = cclw_data.explode('Topic/Response')
cclw_data['Topic/Response'] = cclw_data['Topic/Response'].str.strip()

In [5]:
cclw_data['Topic/Response']

0       Mitigation
1       Mitigation
2       Mitigation
3       Mitigation
4       Mitigation
           ...    
8969           NaN
8970           NaN
8971           NaN
8972           NaN
8973           NaN
Name: Topic/Response, Length: 11139, dtype: object

In [6]:
cclw_data = cclw_data[~cclw_data['Topic/Response'].isna()]
cclw_data

,Document ID,Document Title,Family ID,Family Title,Family Summary,Collection Title(s),Collection Description(s),Document Variant,Document Content URL,Language,...,Document Type,Category,Framework,Topic/Response,Hazard,Sector,Keyword,Instrument,Author,Author Type
0,ordinance-730-03-on-the-encouragement-of-the-p...,Ordinance 730.03 on the encouragement of the p...,ordinance-730-03-on-the-encouragement-of-the-p...,Ordinance 730.03 on the encouragement of the p...,<p>This regulation defines basic determination...,NaN,NaN,Original Language,https://www.fedlex.admin.ch/filestore/fedlex.d...,French;German,...,Regulation,Executive,NaN,Mitigation,NaN,Energy,Renewables;Electricity;Heat,"Processes, plans and strategies|Governance",NaN,NaN
1,green-construction-through-wood-program-applic...,Green Construction through Wood Program - APPL...,green-construction-through-wood-program_d7f2,Green Construction through Wood Program,<p>The Green Construction through Wood (GCWood...,NaN,NaN,Original Language,https://ostr-backend-prod.azurewebsites.net/se...,English,...,Programme,Executive,NaN,Mitigation,NaN,Buildings,Energy Demand;Energy Efficiency;Mitigation,Provision of climate funds|Direct Investment,NaN,NaN
2,product-emissions-standards-act_2677,Product Emissions Standards Act,product-emissions-standards-act-2017_7c97,Product Emissions Standards Act (2017),<p>This act establishes a national framework t...,NaN,NaN,Original Language,https://www.legislation.gov.au/C2017A00104/lat...,English,...,Act,Legislative,NaN,Mitigation,NaN,Industry;Energy;Agriculture;LULUCF,Energy Efficiency;Energy Demand,"Standards, obligations and norms|Regulation",NaN,NaN
3,national-steel-policy_5eeb,National steel policy,national-steel-policy_42a5,National steel policy,<p>the Government has come out with National S...,NaN,NaN,Original Language,https://steel.gov.in/national-steel-policy-nsp...,English,...,Policy,Executive,NaN,Mitigation,NaN,Industry,Mitigation,"Standards, obligations and norms|Regulation;Pr...",NaN,NaN
4,notification-s-o-2670-e-fuel-consumption-stand...,Notification S.O. 2670(E) - Fuel-Consumption S...,fuel-consumption-standards-for-heavy-duty-vehi...,Fuel-Consumption Standards for Heavy-Duty Vehi...,<p>India initially introduced fuel consumption...,NaN,NaN,Original Language,https://www.transportpolicy.net/wp-content/upl...,English;Hindi,...,Rules,Executive,NaN,Mitigation,NaN,Transport,Energy Efficiency,"Standards, obligations and norms|Regulation",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8885,national-environmental-policy-and-strategies_3afb,National Environmental Policy and Strategies,national-environmental-policy-and-strategies_39dc,National Environmental Policy and Strategies,"Contains some references to climate change, gr...",NaN,NaN,NaN,https://climate-laws.org/rails/active_storage/...,English,...,Policy,Executive,NaN,Mitigation,NaN,Energy,Institutions / Administrative Arrangements;Res...,"Processes, plans and strategies|Governance",NaN,NaN
8926,heat-act_b7bd,Heat Act,heat-act-warmtewet-wm_be11,"Heat Act (Warmtewet, Wm)",<p>The Heat Act's last revisions encourage the...,NaN,NaN,NaN,https://climate-laws.org/rails/active_storage/...,Dutch,...,Law,Legislative,NaN,Mitigation,NaN,Energy,Energy Supply;Energy Demand,"Processes, plans and strategies|Governance",NaN,NaN
8948,offshore-petroleum-and-greenhouse-gas-storage-...,Offshore Petroleum and Greenhouse Gas Storage ...,offshore-petroleum-and-greenhouse-gas-storage-...,Offshore Petroleum and Greenhouse Gas Storage ...,The Act aims to provide an effective regulator...,NaN,NaN,NaN,NaN,NaN,...,Law,Legislative,NaN,Mitigation,NaN,Health;Water,Energy Supply,"Processes, plans and strategies|Governance",NaN,NaN
8949,national-climate-change-management-agenda_1c99,National Climate Change Management Agenda,national-climate-change-management-agenda_f98d,National Climate Change Management Agenda,<p>The Agenda is an updated version of the <a ...,NaN,NaN,NaN,https://u

In [7]:
cclw_data.columns

Index(['Document ID', 'Document Title', 'Family ID', 'Family Title',
       'Family Summary', 'Collection Title(s)', 'Collection Description(s)',
       'Document Variant', 'Document Content URL', 'Language', 'Source',
       'Geography ISOs', 'Geographies', 'First event in timeline',
       'Last event in timeline', 'Full timeline of events (types)',
       'Full timeline of events (dates)', 'Date Added to System',
       'Last Modified on System', 'Internal Document ID', 'Internal Family ID',
       'Internal Corpus ID', 'Internal Collection ID(s)', 'Document Role',
       'Document Type', 'Category', 'Framework', 'Topic/Response', 'Hazard',
       'Sector', 'Keyword', 'Instrument', 'Author', 'Author Type'],
      dtype='object')

In [8]:
cclw_data[['Last event in timeline', 'Full timeline of events (types)',
       'Full timeline of events (dates)', 'Date Added to System']]

,Last event in timeline,Full timeline of events (types),Full timeline of events (dates),Date Added to System
0,2025-05-21,Passed/Approved;Passed/Approved,2017-11-01;2025-05-21,2024-09-08
1,2017-10-06,Passed/Approved,2017-10-06,2024-09-08
2,2021-09-01,Passed/Approved;Passed/Approved,2017-09-15;2021-09-01,2024-09-09
3,2017-05-08,Passed/Approved,2017-05-08,2024-09-10
4,2020-09-21,Amended;Passed/Approved,2020-09-21;2017-01-10,2024-09-09
...,...,...,...,...
8885,2003-08-18,Passed/Approved,2003-08-18,2023-11-30
8926,2024-04-23,Passed/Approved;Amended,2009-12-25;2024-04-23,2023-11-30
8948,2008-07-01,Entered Into Force;Passed/Approved,2006-03-29;2008-07-01,2023-11-30
8949,2021-06-30,Passed/Approved,2021-06-30,2023-11-30


In [9]:
# Apply the function row-wise to get the first approved date
cclw_data['Year'] = cclw_data.apply(
    lambda row: get_first_important_year(row['Full timeline of events (types)'], row['Full timeline of events (dates)']), axis=1
)


cclw_data['Year']

0       2017
1       2017
2       2017
3       2017
4       2017
        ... 
8885    2003
8926    2009
8948    2006
8949    2021
8949    2021
Name: Year, Length: 8394, dtype: object

In [10]:
# Now group and count
topic_counts = cclw_data.groupby(['Geography ISOs', 'Year', 'Topic/Response']).size().reset_index(name='Count').rename(columns={'Geography ISOs': 'Country'})
topic_counts

,Country,Year,Topic/Response,Count
0,AFG,2007,Adaptation,1
1,AFG,2007,Mitigation,1
2,AFG,2008,Mitigation,1
3,AFG,2009,Adaptation,1
4,AFG,2011,Adaptation,3
...,...,...,...,...
3735,ZWE,2022,Adaptation,1
3736,ZWE,2022,Disaster Risk Management,1
3737,ZWE,2022,Mitigation,1
3738,ZWE,2023,Adaptation,5


In [11]:
# Pivot to wide format
topic_pivot = topic_counts.pivot_table(index=['Country', 'Year'], 
                                       columns='Topic/Response', 
                                       values='Count', 
                                       fill_value=0).reset_index()

topic_pivot

Topic/Response,Country,Year,Adaptation,Disaster Risk Management,Loss And Damage,Mitigation
0,AFG,2007,1.0,0.0,0.0,1.0
1,AFG,2008,0.0,0.0,0.0,1.0
2,AFG,2009,1.0,0.0,0.0,0.0
3,AFG,2011,3.0,1.0,0.0,2.0
4,AFG,2012,2.0,2.0,0.0,0.0
...,...,...,...,...,...,...
2195,ZWE,2018,3.0,1.0,0.0,1.0
2196,ZWE,2019,3.0,1.0,1.0,1.0
2197,ZWE,2020,4.0,1.0,0.0,6.0
2198,ZWE,2022,1.0,1.0,0.0,1.0


In [12]:
countries = cclw_data['Geography ISOs'].unique().tolist()
countries

['CHE',
 'CAN',
 'AUS',
 'IND',
 'CHL',
 'VNM',
 'FRA',
 'UKR',
 'ESP',
 'DZA',
 'NLD',
 'KEN',
 'PER',
 'SWE',
 'MEX',
 'ARG',
 'ITA',
 'NOR',
 'MLT',
 'ROU',
 'GHA',
 'URY',
 'IDN',
 'PAN',
 'ETH',
 'KOR',
 'ZMB',
 'GTM',
 'JPN',
 'TUR',
 'RUS',
 'WSM',
 'ECU',
 'GIN',
 'MOZ',
 'NIC',
 'BRA',
 'PRT',
 'MDV',
 'SAU',
 'GBR',
 'PHL',
 'MDA',
 'XKX',
 'CHN',
 'CRI',
 'SLV',
 'COM',
 'HTI',
 'COL',
 'SEN',
 'BOL',
 'DEU',
 'COG',
 'CPV',
 'BFA',
 'PRY',
 'NZL',
 'BTN',
 'USA',
 'MAR',
 'COD',
 'NGA',
 'KAZ',
 'NPL',
 'THA',
 'VEN',
 'PNG',
 'ZAF',
 'FSM',
 'GRC',
 'HUN',
 'EST',
 'HRV',
 'LVA',
 'AUT',
 'POL',
 'SVK',
 'MUS',
 'ALB',
 'TWN',
 'BGR',
 'QAT',
 'FIN',
 'CZE',
 'CYP',
 'LTU',
 'SVN',
 'SYC',
 'HND',
 'UGA',
 'BGD',
 'MNG',
 'NER',
 'EUR',
 'IRL',
 'DNK',
 'LKA',
 'JOR',
 'SLE',
 'BWA',
 'MMR',
 'TJK',
 'FJI',
 'MYS',
 'SYR',
 'GEO',
 'AND',
 'IRN',
 'BLR',
 'VUT',
 'TGO',
 'TCD',
 'AGO',
 'SMR',
 'MRT',
 'LAO',
 'ISR',
 'GNQ',
 'SSD',
 'LUX',
 'LIE',
 'KGZ',
 'BLZ',
 'DOM',


In [13]:
wdi_data = pd.read_csv('../data/WDI_Data.csv')
wdi_data

,Country Name,Country Code,Series Name,Series Code,2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022],2023 [YR2023],2024 [YR2024]
0,Brazil,BRA,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,61.779,58.372,56.558,54.287,50.958,47.46,45.67,42.881,42.686,..
1,Brazil,BRA,"Agriculture, forestry, and fishing, value adde...",NV.AGR.TOTL.ZS,4.31914942942436,4.89135358845478,4.60059169575972,4.42039930378329,4.2050140943502,5.71148511544041,6.55876261159667,5.76747490819457,6.24138467625666,..
2,Brazil,BRA,"Annual freshwater withdrawals, total (% of int...",ER.H2O.FWTL.ZS,1.1280692457163,1.17134781840664,1.16021904257198,1.17470411588059,1.19130895601484,1.18689277512807,1.18901254195372,..,..,..
3,Brazil,BRA,Births attended by skilled health staff (% of ...,SH.STA.BRTC.ZS,98.2,98.6,98.7,98.8,98.4,98.4,..,..,..,..
4,Brazil,BRA,"Contraceptive prevalence, any method (% of mar...",SP.DYN.CONU.ZS,..,..,..,..,..,..,..,..,..,..
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13832,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13833,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13834,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13835,Data from database: World Development Indicators,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
wdi_data = wdi_data[wdi_data['Country Code'].isin(countries)]
wdi_data

,Country Name,Country Code,Series Name,Series Code,2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022],2023 [YR2023],2024 [YR2024]
0,Brazil,BRA,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,61.779,58.372,56.558,54.287,50.958,47.46,45.67,42.881,42.686,..
1,Brazil,BRA,"Agriculture, forestry, and fishing, value adde...",NV.AGR.TOTL.ZS,4.31914942942436,4.89135358845478,4.60059169575972,4.42039930378329,4.2050140943502,5.71148511544041,6.55876261159667,5.76747490819457,6.24138467625666,..
2,Brazil,BRA,"Annual freshwater withdrawals, total (% of int...",ER.H2O.FWTL.ZS,1.1280692457163,1.17134781840664,1.16021904257198,1.17470411588059,1.19130895601484,1.18689277512807,1.18901254195372,..,..,..
3,Brazil,BRA,Births attended by skilled health staff (% of ...,SH.STA.BRTC.ZS,98.2,98.6,98.7,98.8,98.4,98.4,..,..,..,..
4,Brazil,BRA,"Contraceptive prevalence, any method (% of mar...",SP.DYN.CONU.ZS,..,..,..,..,..,..,..,..,..,..
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11279,Zimbabwe,ZWE,Surface area (sq. km),AG.SRF.TOTL.K2,390760,390760,390760,390760,390760,390760,390760,390760,..,..
11280,Zimbabwe,ZWE,Tax revenue (% of GDP),GC.TAX.TOTL.GD.ZS,17.6733631953313,15.4583408165803,5.46551150806074,7.21476500513885,..,..,..,..,..,..
11281,Zimbabwe,ZWE,Terrestrial and marine protected areas (% of t...,ER.PTD.TOTL.ZS,26.7,27.2145424089062,27.2,27.2145851759137,27.2,27.2,27.2,27.2,28.3,28.3
11282,Zimbabwe,ZWE,"Total debt service (% of exports of goods, ser...",DT.TDS.DECT.EX.ZS,16.8000177011432,30.5852406243532,15.4997229691463,11.6808018902759,30.2372345236683,18.6816776876239,9.22921667100653,6.08864103314875,15.2693687383439,..


In [15]:
wdi_data['Series Name'].value_counts()

Series Name
Adolescent fertility rate (births per 1,000 women ages 15-19)                  196
Agriculture, forestry, and fishing, value added (% of GDP)                     196
Annual freshwater withdrawals, total (% of internal resources)                 196
Births attended by skilled health staff (% of total)                           196
Contraceptive prevalence, any method (% of married women ages 15-49)           196
Domestic credit provided by financial sector (% of GDP)                        196
Electric power consumption (kWh per capita)                                    196
Energy use (kg of oil equivalent per capita)                                   196
Exports of goods and services (% of GDP)                                       196
External debt stocks, total (DOD, current US$)                                 196
Fertility rate, total (births per woman)                                       196
Foreign direct investment, net inflows (BoP, current US$)                  

In [16]:
wdi_data[wdi_data['Series Name'] == 'Mortality rate, under-5 (per 1,000 live births)'].isna().sum()## NaN values are not recognized

Country Name     0
Country Code     0
Series Name      0
Series Code      0
2015 [YR2015]    0
2016 [YR2016]    0
2017 [YR2017]    0
2018 [YR2018]    0
2019 [YR2019]    0
2020 [YR2020]    0
2021 [YR2021]    0
2022 [YR2022]    0
2023 [YR2023]    0
2024 [YR2024]    0
dtype: int64

The numeric columns (2015 onwards) have ```dtype('0')``` because of mixed type of values. Primarily, it is because the ```NaN``` (missing values) are actually represented as strings. We ensure first that these convertible values are turned to ```NaN```.

In [17]:
wdi_data.iloc[:, 4:] = wdi_data.iloc[:, 4:].apply(pd.to_numeric, errors = 'coerce')

/tmp/ipykernel_660252/338750357.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wdi_data.iloc[:, 4:] = wdi_data.iloc[:, 4:].apply(pd.to_numeric, errors = 'coerce')


In [18]:
wdi_data[wdi_data['Series Name'] == 'Prevalence of underweight, weight for age (% of children under 5)']['2016 [YR2016]']#checking

41        NaN
93        NaN
145       0.6
197       NaN
249       NaN
         ... 
11013    13.8
11117     NaN
11169     NaN
11221     NaN
11273     NaN
Name: 2016 [YR2016], Length: 196, dtype: object

Checking all the variables we are shortlisted in at this point.

In [19]:
wdi_data[wdi_data['Series Name'] == 'Mortality rate, under-5 (per 1,000 live births)'].iloc[:, 4:].isna().sum() #yes

2015 [YR2015]      2
2016 [YR2016]      2
2017 [YR2017]      2
2018 [YR2018]      2
2019 [YR2019]      2
2020 [YR2020]      2
2021 [YR2021]      2
2022 [YR2022]      2
2023 [YR2023]      2
2024 [YR2024]    196
dtype: int64

In [20]:
wdi_data[wdi_data['Series Name'] == 'Prevalence of underweight, weight for age (% of children under 5)'].iloc[:, 4:].isna().sum() #no

2015 [YR2015]    165
2016 [YR2016]    160
2017 [YR2017]    168
2018 [YR2018]    150
2019 [YR2019]    151
2020 [YR2020]    173
2021 [YR2021]    166
2022 [YR2022]    168
2023 [YR2023]    174
2024 [YR2024]    190
dtype: int64

In [21]:
wdi_data[wdi_data['Series Name'] == 'Immunization, measles (% of children ages 12-23 months)'].iloc[:, 4:].isna().sum() #yes

2015 [YR2015]      3
2016 [YR2016]      3
2017 [YR2017]      3
2018 [YR2018]      3
2019 [YR2019]      3
2020 [YR2020]      3
2021 [YR2021]      3
2022 [YR2022]      3
2023 [YR2023]      3
2024 [YR2024]    196
dtype: int64

In [22]:
wdi_data[wdi_data['Series Name'] == 'Life expectancy at birth, total (years)'].iloc[:, 4:].isna().sum() # yes

2015 [YR2015]      0
2016 [YR2016]      0
2017 [YR2017]      0
2018 [YR2018]      0
2019 [YR2019]      0
2020 [YR2020]      0
2021 [YR2021]      0
2022 [YR2022]      0
2023 [YR2023]      0
2024 [YR2024]    196
dtype: int64

In [23]:
wdi_data[wdi_data['Series Name'] == 'Births attended by skilled health staff (% of total)'].iloc[:, 4:].isna().sum() # no

2015 [YR2015]     95
2016 [YR2016]     95
2017 [YR2017]    101
2018 [YR2018]    100
2019 [YR2019]    110
2020 [YR2020]    140
2021 [YR2021]    170
2022 [YR2022]    193
2023 [YR2023]    196
2024 [YR2024]    196
dtype: int64

In [24]:
wdi_data[wdi_data['Series Name'] == 'Fertility rate, total (births per woman)'].iloc[:, 4:].isna().sum()#yes

2015 [YR2015]      0
2016 [YR2016]      0
2017 [YR2017]      0
2018 [YR2018]      0
2019 [YR2019]      0
2020 [YR2020]      0
2021 [YR2021]      0
2022 [YR2022]      0
2023 [YR2023]      0
2024 [YR2024]    196
dtype: int64

In [25]:
wdi_data[wdi_data['Series Name'] == 'Poverty headcount ratio at $3.00 a day (2021 PPP) (% of population)'].iloc[:, 4:].isna().sum() # no #SI.POV.DDAY

2015 [YR2015]    110
2016 [YR2016]    113
2017 [YR2017]    118
2018 [YR2018]    102
2019 [YR2019]    118
2020 [YR2020]    128
2021 [YR2021]    119
2022 [YR2022]    135
2023 [YR2023]    172
2024 [YR2024]    194
dtype: int64

In [26]:
wdi_data[wdi_data['Series Name'] == 'Population growth (annual %)'].iloc[:, 4:].isna().sum() # yes

2015 [YR2015]      0
2016 [YR2016]      0
2017 [YR2017]      0
2018 [YR2018]      0
2019 [YR2019]      0
2020 [YR2020]      0
2021 [YR2021]      0
2022 [YR2022]      1
2023 [YR2023]      0
2024 [YR2024]    196
dtype: int64

In [27]:
wdi_data[wdi_data['Series Name'] == 'Poverty headcount ratio at $3.00 a day (2021 PPP) (% of population)'].iloc[:, 4:].isna().sum() # no #SI.POV.DDAY

2015 [YR2015]    110
2016 [YR2016]    113
2017 [YR2017]    118
2018 [YR2018]    102
2019 [YR2019]    118
2020 [YR2020]    128
2021 [YR2021]    119
2022 [YR2022]    135
2023 [YR2023]    172
2024 [YR2024]    194
dtype: int64

In [28]:
wdi_data[wdi_data['Series Name'] == 'Population density (people per sq. km of land area)'].iloc[:, 4:].isna().sum() # yes, because most of the values from earlier than 2022 are present

2015 [YR2015]      1
2016 [YR2016]      1
2017 [YR2017]      1
2018 [YR2018]      1
2019 [YR2019]      1
2020 [YR2020]      1
2021 [YR2021]      1
2022 [YR2022]      4
2023 [YR2023]    196
2024 [YR2024]    196
dtype: int64

In [29]:
# Melt the wide format to long format, since we want one row per (country, year, series) (long/panel format).
health_df = pd.melt(
    wdi_data,
    id_vars=['Country Name', 'Country Code', 'Series Name', 'Series Code'],
    var_name='Year',
    value_name='Value'
)

health_df

,Country Name,Country Code,Series Name,Series Code,Year,Value
0,Brazil,BRA,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,2015 [YR2015],61.779
1,Brazil,BRA,"Agriculture, forestry, and fishing, value adde...",NV.AGR.TOTL.ZS,2015 [YR2015],4.319149
2,Brazil,BRA,"Annual freshwater withdrawals, total (% of int...",ER.H2O.FWTL.ZS,2015 [YR2015],1.128069
3,Brazil,BRA,Births attended by skilled health staff (% of ...,SH.STA.BRTC.ZS,2015 [YR2015],98.2
4,Brazil,BRA,"Contraceptive prevalence, any method (% of mar...",SP.DYN.CONU.ZS,2015 [YR2015],NaN
...,...,...,...,...,...,...
101915,Zimbabwe,ZWE,Surface area (sq. km),AG.SRF.TOTL.K2,2024 [YR2024],NaN
101916,Zimbabwe,ZWE,Tax revenue (% of GDP),GC.TAX.TOTL.GD.ZS,2024 [YR2024],NaN
101917,Zimbabwe,ZWE,Terrestrial and marine protected areas (% of t...,ER.PTD.TOTL.ZS,2024 [YR2024],28.3
101918,Zimbabwe,ZWE,"Total debt service (% of exports of goods, ser...",DT.TDS.DECT.EX.ZS,2024 [YR2024],NaN


In [30]:
# Extract the numeric year (first 4 digits) using regex
health_df['Year'] = health_df['Year'].str.extract(r'(\d{4})').astype(int)

In [31]:
## Before merging, change the Country code column to Country code ISO  naming it 'Country' for simplicity
health_df = health_df.rename(columns={'Country Code': 'Country'})
health_df

,Country Name,Country,Series Name,Series Code,Year,Value
0,Brazil,BRA,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,2015,61.779
1,Brazil,BRA,"Agriculture, forestry, and fishing, value adde...",NV.AGR.TOTL.ZS,2015,4.319149
2,Brazil,BRA,"Annual freshwater withdrawals, total (% of int...",ER.H2O.FWTL.ZS,2015,1.128069
3,Brazil,BRA,Births attended by skilled health staff (% of ...,SH.STA.BRTC.ZS,2015,98.2
4,Brazil,BRA,"Contraceptive prevalence, any method (% of mar...",SP.DYN.CONU.ZS,2015,NaN
...,...,...,...,...,...,...
101915,Zimbabwe,ZWE,Surface area (sq. km),AG.SRF.TOTL.K2,2024,NaN
101916,Zimbabwe,ZWE,Tax revenue (% of GDP),GC.TAX.TOTL.GD.ZS,2024,NaN
101917,Zimbabwe,ZWE,Terrestrial and marine protected areas (% of t...,ER.PTD.TOTL.ZS,2024,28.3
101918,Zimbabwe,ZWE,"Total debt service (% of exports of goods, ser...",DT.TDS.DECT.EX.ZS,2024,NaN


In [32]:
# Remove the rows where value is NaN
health_df = health_df.dropna(subset=['Value'])
health_df

,Country Name,Country,Series Name,Series Code,Year,Value
0,Brazil,BRA,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,2015,61.779
1,Brazil,BRA,"Agriculture, forestry, and fishing, value adde...",NV.AGR.TOTL.ZS,2015,4.319149
2,Brazil,BRA,"Annual freshwater withdrawals, total (% of int...",ER.H2O.FWTL.ZS,2015,1.128069
3,Brazil,BRA,Births attended by skilled health staff (% of ...,SH.STA.BRTC.ZS,2015,98.2
6,Brazil,BRA,Electric power consumption (kWh per capita),EG.USE.ELEC.KH.PC,2015,2628.097691
...,...,...,...,...,...,...
101848,Zambia,ZMB,Net migration,SM.POP.NETM,2024,7381.0
101865,Zambia,ZMB,Terrestrial and marine protected areas (% of t...,ER.PTD.TOTL.ZS,2024,41.3
101900,Zimbabwe,ZWE,Net migration,SM.POP.NETM,2024,-60528.0
101909,Zimbabwe,ZWE,"Prevalence of underweight, weight for age (% o...",SH.STA.MALN.ZS,2024,9.6


In [ ]:
## Skipped this step after finding no significance in the following variables
'''# Since the 'Series Name' is too long, I am going to use the Series Code to arrange the table from now
codes_of_interest = ['SH.DYN.MORT', 'SH.IMM.MEAS', 'SP.DYN.LE00.IN', 'SP.DYN.TFRT.IN', 'EN.POP.DNST', 'SP.POP.GROW']

# Select to keep the rows we are interested in, based on the series code
health_df = health_df[health_df['Series Code'].isin(codes_of_interest)]
health_df'''

,Country Name,Country,Series Name,Series Code,Year,Value
10,Brazil,BRA,"Fertility rate, total (births per woman)",SP.DYN.TFRT.IN,2015,1.777
21,Brazil,BRA,"Immunization, measles (% of children ages 12-2...",SH.IMM.MEAS,2015,96.0
26,Brazil,BRA,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,2015,75.106
30,Brazil,BRA,"Mortality rate, under-5 (per 1,000 live births)",SH.DYN.MORT,2015,16.0
35,Brazil,BRA,Population density (people per sq. km of land ...,EN.POP.DNST,2015,24.129236
...,...,...,...,...,...,...
91686,Zimbabwe,ZWE,"Fertility rate, total (births per woman)",SP.DYN.TFRT.IN,2023,3.724
91697,Zimbabwe,ZWE,"Immunization, measles (% of children ages 12-2...",SH.IMM.MEAS,2023,90.0
91702,Zimbabwe,ZWE,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,2023,62.775
91706,Zimbabwe,ZWE,"Mortality rate, under-5 (per 1,000 live births)",SH.DYN.MORT,2023,44.2


In [49]:
# Step 1: Pivot the data to get Series Names as columns
panel_df = health_df.pivot_table(
    index=['Country', 'Year'],
    columns='Series Code',
    values='Value'
).reset_index()
panel_df

Series Code,Country,Year,AG.LND.FRST.K2,AG.SRF.TOTL.K2,BX.KLT.DINV.CD.WD,BX.TRF.PWKR.CD.DT,DT.DOD.DECT.CD,DT.ODA.ALLD.CD,DT.TDS.DECT.EX.ZS,EG.USE.ELEC.KH.PC,...,SP.ADO.TFRT,SP.DYN.CONU.ZS,SP.DYN.LE00.IN,SP.DYN.TFRT.IN,SP.POP.GROW,SP.POP.TOTL,SP.URB.GROW,TG.VAL.TOTL.GD.ZS,TT.PRI.MRCH.XD.WD,TX.VAL.TECH.MF.ZS
0,AFG,2015,12084.4,652860.0,169146608.0,348624717.302647,2596917266.2,4274229980.46875,3.299608,NaN,...,81.043,22.5,62.27,5.652,3.119959,33831764.0,3.994626,43.34642,100.0,NaN
1,AFG,2016,12084.4,652860.0,93591315.3,627710802.377386,2596050086.4,4069409912.10938,4.333403,NaN,...,78.13,NaN,62.646,5.542,2.53572,34700612.0,3.406813,39.356231,106.302534,NaN
2,AFG,2017,12084.4,652860.0,51533896.765,822731630.245558,2751986638.9,3811739990.23438,4.353572,NaN,...,75.3,NaN,62.406,5.433,2.808337,35688935.0,3.723403,41.528344,108.415252,NaN
3,AFG,2018,12084.4,652860.0,119435105.715177,803546453.786557,2678760424.2,3792489990.23438,3.3238,NaN,...,73.021,NaN,62.443,5.327,2.91081,36743039.0,3.876433,45.875466,105.630246,0.0
4,AFG,2019,12084.4,652860.0,23404553.647249,828571904.111179,2661685956.5,4136990234.375,2.974238,NaN,...,70.967,NaN,62.941,5.238,2.984389,37856121.0,3.995143,40.644818,111.903426,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1955,ZWE,2020,174445.8,390760.0,150360000.0,1832039380.8915,12742029136.9,985049987.792969,18.681678,464.806599,...,98.893,NaN,61.53,3.754,1.659353,15526888.0,1.758639,37.360517,109.417727,2.383868
1956,ZWE,2021,173985.1,390760.0,237480367.76,2573626885.68752,13817579341.299999,985789978.027344,9.229217,540.095371,...,99.199,NaN,60.135,3.765,1.726011,15797210.0,1.915036,49.973357,97.836232,1.05374
1957,ZWE,2022,173524.4,390760.0,336761000.0,3085043420.98222,13829538551.1,782289978.027344,6.088641,538.799541,...,99.484,NaN,62.36,3.767,1.706209,16069056.0,1.990609,46.474887,NaN,1.476931
1958,ZWE,2023,NaN,NaN,557873000.0,3301267763.74879,14213387010.700001,NaN,15.269369,NaN,...,98.057,NaN,62.775,3.724,1.677096,16340822.0,2.052982,46.660124,NaN,1.942188


In [50]:
panel_df = pd.merge(
    panel_df,
    topic_pivot,
    on=['Country', 'Year'],
    how='inner'  # Or 'left' if you want to keep all health data
)
panel_df

,Country,Year,AG.LND.FRST.K2,AG.SRF.TOTL.K2,BX.KLT.DINV.CD.WD,BX.TRF.PWKR.CD.DT,DT.DOD.DECT.CD,DT.ODA.ALLD.CD,DT.TDS.DECT.EX.ZS,EG.USE.ELEC.KH.PC,...,SP.POP.GROW,SP.POP.TOTL,SP.URB.GROW,TG.VAL.TOTL.GD.ZS,TT.PRI.MRCH.XD.WD,TX.VAL.TECH.MF.ZS,Adaptation,Disaster Risk Management,Loss And Damage,Mitigation
0,AFG,2015,12084.4,652860.0,169146608.0,348624717.302647,2596917266.2,4274229980.46875,3.299608,NaN,...,3.119959,33831764.0,3.994626,43.34642,100.0,NaN,2.0,0.0,0.0,3.0
1,AFG,2016,12084.4,652860.0,93591315.3,627710802.377386,2596050086.4,4069409912.10938,4.333403,NaN,...,2.53572,34700612.0,3.406813,39.356231,106.302534,NaN,1.0,0.0,0.0,0.0
2,AFG,2017,12084.4,652860.0,51533896.765,822731630.245558,2751986638.9,3811739990.23438,4.353572,NaN,...,2.808337,35688935.0,3.723403,41.528344,108.415252,NaN,1.0,1.0,0.0,1.0
3,AGO,2015,693826.9,1246700.0,10028215162.6394,11114711.6,48771532654.300003,380059997.558594,24.785302,306.167407,...,3.605072,28157798.0,4.738415,59.531636,100.0,12.604486,0.0,0.0,0.0,1.0
4,AGO,2016,688276.2,1246700.0,-179517618.92,3988048.38,57434097528.199997,206509994.506836,38.075644,331.6649,...,3.576441,29183070.0,4.678376,77.004842,87.893416,12.988305,2.0,2.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
998,ZWE,2018,175367.2,390760.0,717865322.24885,1427703019.441,12689056308.4,794559997.558594,11.680802,641.65957,...,1.487416,15034452.0,1.400526,30.588999,100.233916,2.063693,3.0,1.0,0.0,1.0
999,ZWE,2019,174906.5,390760.0,249500000.0,1417011952.8873,12249359646.4,844090026.855469,30.237235,520.778492,...,1.563534,15271368.0,1.566638,35.330152,103.430712,4.949714,3.0,1.0,1.0,1.0
1000,ZWE,2020,174445.8,390760.0,150360000.0,1832039380.8915,12742029136.9,985049987.792969,18.681678,464.806599,...,1.659353,15526888.0,1.758639,37.360517,109.417727,2.383868,4.0,1.0,0.0,6.0
1001,ZWE,2022,173524.4,390760.0,336761000.0,3085043420.98222,13829538551.1,782289978.027344,6.088641,538.799541,...,1.706209,16069056.0,1.990609,46.474887,NaN,1.476931,1.0,1.0,0.0,1.0


In [51]:
panel_df.columns

Index(['Country', 'Year', 'AG.LND.FRST.K2', 'AG.SRF.TOTL.K2',
       'BX.KLT.DINV.CD.WD', 'BX.TRF.PWKR.CD.DT', 'DT.DOD.DECT.CD',
       'DT.ODA.ALLD.CD', 'DT.TDS.DECT.EX.ZS', 'EG.USE.ELEC.KH.PC',
       'EG.USE.PCAP.KG.OE', 'EN.POP.DNST', 'ER.H2O.FWTL.ZS', 'ER.PTD.TOTL.ZS',
       'FS.AST.DOMS.GD.ZS', 'GC.REV.XGRT.GD.ZS', 'GC.TAX.TOTL.GD.ZS',
       'IT.CEL.SETS.P2', 'MS.MIL.XPND.GD.ZS', 'NE.EXP.GNFS.ZS',
       'NE.GDI.TOTL.ZS', 'NE.IMP.GNFS.ZS', 'NV.AGR.TOTL.ZS', 'NV.IND.TOTL.ZS',
       'NY.GDP.DEFL.KD.ZG', 'NY.GDP.MKTP.CD', 'NY.GDP.MKTP.KD.ZG',
       'NY.GNP.ATLS.CD', 'NY.GNP.MKTP.PP.CD', 'NY.GNP.PCAP.CD',
       'NY.GNP.PCAP.PP.CD', 'SE.ENR.PRSC.FM.ZS', 'SE.PRM.CMPT.ZS',
       'SE.PRM.ENRR', 'SE.SEC.ENRR', 'SH.DYN.AIDS.ZS', 'SH.DYN.MORT',
       'SH.IMM.MEAS', 'SH.STA.BRTC.ZS', 'SH.STA.MALN.ZS', 'SI.DST.FRST.20',
       'SI.POV.DDAY', 'SI.POV.NAHC', 'SM.POP.NETM', 'SP.ADO.TFRT',
       'SP.DYN.CONU.ZS', 'SP.DYN.LE00.IN', 'SP.DYN.TFRT.IN', 'SP.POP.GROW',
       'SP.POP.TOTL', 'SP

In [52]:
# To rename the columns from the codes to an understandable variable name
## Skipped at this stage
'''series_codes = {
    'SH.DYN.MORT': 'Under5_Mortality',
    'SH.IMM.MEAS': 'Measles_Immunization',
    'SP.DYN.LE00.IN': 'Life_Expectancy',
    'SP.DYN.TFRT.IN': 'Fertility_Rate',
    'EN.POP.DNST': 'Pop_Density',
    'SP.POP.GROW': 'Pop_Growth'
}

panel_df = panel_df.rename(columns=series_codes)
panel_df'''


"series_codes = {\n    'SH.DYN.MORT': 'Under5_Mortality',\n    'SH.IMM.MEAS': 'Measles_Immunization',\n    'SP.DYN.LE00.IN': 'Life_Expectancy',\n    'SP.DYN.TFRT.IN': 'Fertility_Rate',\n    'EN.POP.DNST': 'Pop_Density',\n    'SP.POP.GROW': 'Pop_Growth'\n}\n\npanel_df = panel_df.rename(columns=series_codes)\npanel_df"

In [53]:
# Ensure one unique row per Country
country_names = health_df[['Country Name', 'Country']].drop_duplicates(subset='Country')

# Adding back the country names in case we need it for later
panel_df = panel_df.drop(columns='Country Name', errors='ignore')  # remove before merge if already exists
panel_df = pd.merge(panel_df, country_names, on='Country', how='left')

panel_df

,Country,Year,AG.LND.FRST.K2,AG.SRF.TOTL.K2,BX.KLT.DINV.CD.WD,BX.TRF.PWKR.CD.DT,DT.DOD.DECT.CD,DT.ODA.ALLD.CD,DT.TDS.DECT.EX.ZS,EG.USE.ELEC.KH.PC,...,SP.POP.TOTL,SP.URB.GROW,TG.VAL.TOTL.GD.ZS,TT.PRI.MRCH.XD.WD,TX.VAL.TECH.MF.ZS,Adaptation,Disaster Risk Management,Loss And Damage,Mitigation,Country Name
0,AFG,2015,12084.4,652860.0,169146608.0,348624717.302647,2596917266.2,4274229980.46875,3.299608,NaN,...,33831764.0,3.994626,43.34642,100.0,NaN,2.0,0.0,0.0,3.0,Afghanistan
1,AFG,2016,12084.4,652860.0,93591315.3,627710802.377386,2596050086.4,4069409912.10938,4.333403,NaN,...,34700612.0,3.406813,39.356231,106.302534,NaN,1.0,0.0,0.0,0.0,Afghanistan
2,AFG,2017,12084.4,652860.0,51533896.765,822731630.245558,2751986638.9,3811739990.23438,4.353572,NaN,...,35688935.0,3.723403,41.528344,108.415252,NaN,1.0,1.0,0.0,1.0,Afghanistan
3,AGO,2015,693826.9,1246700.0,10028215162.6394,11114711.6,48771532654.300003,380059997.558594,24.785302,306.167407,...,28157798.0,4.738415,59.531636,100.0,12.604486,0.0,0.0,0.0,1.0,Angola
4,AGO,2016,688276.2,1246700.0,-179517618.92,3988048.38,57434097528.199997,206509994.506836,38.075644,331.6649,...,29183070.0,4.678376,77.004842,87.893416,12.988305,2.0,2.0,0.0,0.0,Angola
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
998,ZWE,2018,175367.2,390760.0,717865322.24885,1427703019.441,12689056308.4,794559997.558594,11.680802,641.65957,...,15034452.0,1.400526,30.588999,100.233916,2.063693,3.0,1.0,0.0,1.0,Zimbabwe
999,ZWE,2019,174906.5,390760.0,249500000.0,1417011952.8873,12249359646.4,844090026.855469,30.237235,520.778492,...,15271368.0,1.566638,35.330152,103.430712,4.949714,3.0,1.0,1.0,1.0,Zimbabwe
1000,ZWE,2020,174445.8,390760.0,150360000.0,1832039380.8915,12742029136.9,985049987.792969,18.681678,464.806599,...,15526888.0,1.758639,37.360517,109.417727,2.383868,4.0,1.0,0.0,6.0,Zimbabwe
1001,ZWE,2022,173524.4,390760.0,336761000.0,3085043420.98222,13829538551.1,782289978.027344,6.088641,538.799541,...,16069056.0,1.990609,46.474887,NaN,1.476931,1.0,1.0,0.0,1.0,Zimbabwe


In [ ]:
# Just to rearrange the country name

cols = list(panel_df.columns)
cols = [cols[-1]] + cols[:-1]
panel_df = panel_df[cols]

In [ ]:
## Renaming columns to avoid spaces
panel_df = panel_df.rename(columns={'Disaster Risk Management': 'Disaster_Risk_Management', 'Loss And Damage': 'Loss_and_Damage'})

In [56]:
panel_df

,Country Name,Country,Year,AG.LND.FRST.K2,AG.SRF.TOTL.K2,BX.KLT.DINV.CD.WD,BX.TRF.PWKR.CD.DT,DT.DOD.DECT.CD,DT.ODA.ALLD.CD,DT.TDS.DECT.EX.ZS,...,SP.POP.GROW,SP.POP.TOTL,SP.URB.GROW,TG.VAL.TOTL.GD.ZS,TT.PRI.MRCH.XD.WD,TX.VAL.TECH.MF.ZS,Adaptation,Disaster_Risk_Management,Loss_and_Damage,Mitigation
0,Afghanistan,AFG,2015,12084.4,652860.0,169146608.0,348624717.302647,2596917266.2,4274229980.46875,3.299608,...,3.119959,33831764.0,3.994626,43.34642,100.0,NaN,2.0,0.0,0.0,3.0
1,Afghanistan,AFG,2016,12084.4,652860.0,93591315.3,627710802.377386,2596050086.4,4069409912.10938,4.333403,...,2.53572,34700612.0,3.406813,39.356231,106.302534,NaN,1.0,0.0,0.0,0.0
2,Afghanistan,AFG,2017,12084.4,652860.0,51533896.765,822731630.245558,2751986638.9,3811739990.23438,4.353572,...,2.808337,35688935.0,3.723403,41.528344,108.415252,NaN,1.0,1.0,0.0,1.0
3,Angola,AGO,2015,693826.9,1246700.0,10028215162.6394,11114711.6,48771532654.300003,380059997.558594,24.785302,...,3.605072,28157798.0,4.738415,59.531636,100.0,12.604486,0.0,0.0,0.0,1.0
4,Angola,AGO,2016,688276.2,1246700.0,-179517618.92,3988048.38,57434097528.199997,206509994.506836,38.075644,...,3.576441,29183070.0,4.678376,77.004842,87.893416,12.988305,2.0,2.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
998,Zimbabwe,ZWE,2018,175367.2,390760.0,717865322.24885,1427703019.441,12689056308.4,794559997.558594,11.680802,...,1.487416,15034452.0,1.400526,30.588999,100.233916,2.063693,3.0,1.0,0.0,1.0
999,Zimbabwe,ZWE,2019,174906.5,390760.0,249500000.0,1417011952.8873,12249359646.4,844090026.855469,30.237235,...,1.563534,15271368.0,1.566638,35.330152,103.430712,4.949714,3.0,1.0,1.0,1.0
1000,Zimbabwe,ZWE,2020,174445.8,390760.0,150360000.0,1832039380.8915,12742029136.9,985049987.792969,18.681678,...,1.659353,15526888.0,1.758639,37.360517,109.417727,2.383868,4.0,1.0,0.0,6.0
1001,Zimbabwe,ZWE,2022,173524.4,390760.0,336761000.0,3085043420.98222,13829538551.1,782289978.027344,6.088641,...,1.706209,16069056.0,1.990609,46.474887,NaN,1.476931,1.0,1.0,0.0,1.0


In [57]:
## Save the dataset for hypothesis testing

panel_df.to_csv('../output_data/panel_dataset.csv', index=False)